In [4]:

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import model_selection, neighbors
import re
from collections import Counter
from string import punctuation
from collections import defaultdict
import datetime
col_Names =["userID", "gender", "age", "occupation", "starSign", "date", "text", "len_text", 
            "capitalized_letters", "ratio_capital_total", "ratio_spaces_total" , "ratio_spaces_non_spaces",
            "ratio_capital_lower_case" ,"ratio_numeric_total", "special_chars", "count_once", "count_twice", 
            "avg_len", "max_len", "words_with_numbers", "words_shorter_three_letters", "ratio_distinct_words_total", 
            "sum_punctuation", "ratio_punctuation_text", "hypen_words", "avg_sen_len_words", 
            "avg_sen_len_chars", "class"]
features = [  "len_text",
            "capitalized_letters", "ratio_capital_total", "ratio_spaces_total" , "ratio_spaces_non_spaces",
            "ratio_capital_lower_case" ,"ratio_numeric_total", "special_chars", "count_once", "count_twice", 
            "avg_len", "max_len", "words_with_numbers", "words_shorter_three_letters", "ratio_distinct_words_total", 
            "sum_punctuation", "ratio_punctuation_text", "hypen_words", "avg_sen_len_words", 
            "avg_sen_len_chars","fre"]

def syllable_count(word):
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("e"):
                count -= 1
    if count == 0:
        count += 1
    return count




def write_csv(csv_file, output_file):
    df = pd.read_csv(csv_file, names=col_Names, dtype = str)

    count=-1
    remove_rows=[]
    for row in df.itertuples():
        count+=1
        if int(row.age) in [n for n in range(14,17)]:
            df.at[row[0],'class'] = '14-16'
        elif int(row.age) in [n for n in range(24,27)]:
            df.at[row[0],'class'] = '24-26'
        elif int(row.age) in [n for n in range(34,37)]:
            df.at[row[0],'class'] = '34-36'
        elif int(row.age) in [n for n in range(44,47)]:
            df.at[row[0],'class'] = '44-46'
            #for testing file we can;t remove because it will decrease our accuracy as number of instance we check become less/
        else:
            
            
            remove_rows.append(count)
            continue
        text = str(row.text.strip())
        sents = text.split('.')
        avg_sen_len_words = sum(len(x.split()) for x in sents) / len(sents)
        avg_sen_len_chars = sum(len(x) for x in sents) / len(sents)
        hypen_words = len(re.findall(r'\w+(?:-\w+)+',text))
        count_once = 0
        count_twice = 0
        sum_len = 0
        max_len = 0
        words_with_numbers = 0
        words_shorter_three_letters = 0
        len_text = len(text)
        if len_text == 0:
            df.at[row[0],'text'] = ''
            len_text=0
        
       
        capitalized_letters = len(re.findall(r'[A-Z]',text))
        lower_case_letters = sum(map(str.islower, text))
        if len_text == 0:
            remove_rows.append(count)
            continue
        ratio_capital_total = round(capitalized_letters / len_text,3)
        spaces = text.count(' ')
        if spaces == len_text:
            remove_rows.append(count)
            continue

        sum_punctuation = sum(Counter(c for c in text if c in punctuation).values())
        for key, val in Counter(text.split()).items():
            if len(key) < 3:
                words_shorter_three_letters += 1
            matches = re.findall('[0-9]', key)
            if len(matches) > 0:
                words_with_numbers += 1
            sum_len += len(key)
            if len(key) >= max_len:
                max_len = len(key)
            if val == 1:
                count_once +=1
            elif val == 2:
                count_twice += 1

        ratio_spaces_total = round(spaces / len_text,3)
        ratio_spaces_non_spaces = round(spaces / (len_text - spaces),3)
        if lower_case_letters == 0:
            lower_case_letters = 1
        ratio_capital_lower_case = round(capitalized_letters / (lower_case_letters),3)
        numeric = sum(c.isdigit() for c in text)
        ratio_numeric_total = round(numeric / len_text,3)
        special_chars = len(text) - spaces - len( re.findall('[\w]', text) )
        ratio_punctuation_text = round(sum_punctuation / len_text, 3)
        num_of_words = len(text.split())
        avg_len = round(sum_len / num_of_words,3)
        ratio_distinct_words_total = round(count_once / num_of_words, 3)
        arg1 = 206.835
        arg2 = 1.015 * (num_of_words / len(sents))
        arg3 = 84.6 * (syllable_count(text) / num_of_words)

        fre = arg1 -arg2 -arg3
        fre1=fre
        
        if(fre1>=121.22):
            fre1=2
        d={1:"14-16",2:"24-26",3:"34-36",4:"44-46"}
        if 90<=fre1:
            fre1 = 1
            
        elif 30<=fre1<90:
            fre1 = 2
        elif 10<fre1<30:
            fre1=3
        else:
            fre1=4
        
        df.at[row[0],'len_text'] = len_text
        df.at[row[0],'capitalized_letters'] = capitalized_letters
        df.at[row[0],'ratio_capital_total'] = ratio_capital_total*100
        df.at[row[0],'ratio_spaces_total'] = ratio_spaces_total*100
        df.at[row[0],'ratio_spaces_non_spaces'] = ratio_spaces_non_spaces*100
        df.at[row[0],'ratio_capital_lower_case'] = ratio_capital_lower_case*100
        df.at[row[0],'ratio_numeric_total'] = ratio_numeric_total*100
        df.at[row[0],'special_chars'] = special_chars
        df.at[row[0],'count_once'] = count_once
        df.at[row[0],'count_twice'] = count_twice
        df.at[row[0],'avg_len'] = avg_len
        df.at[row[0],'max_len'] = max_len
        df.at[row[0],'words_with_numbers'] = words_with_numbers
        df.at[row[0],'words_shorter_three_letters'] = words_shorter_three_letters
        df.at[row[0],'ratio_distinct_words_total'] = ratio_distinct_words_total*100
        df.at[row[0],'sum_punctuation'] = sum_punctuation
        df.at[row[0],'ratio_punctuation_text'] = ratio_punctuation_text*100
        df.at[row[0],'hypen_words'] = hypen_words
        df.at[row[0],'avg_sen_len_words'] = avg_sen_len_words
        df.at[row[0],'avg_sen_len_chars'] = avg_sen_len_chars
        df.at[row[0],'fre'] = fre
    count = 0
    col_scores=[]
 
    indexes_to_keep = set(range(df.shape[0]))-set(remove_rows)
   
    df1=df.take(list(indexes_to_keep))
    print(df1.describe())
    #last column for age
    y= df1['class']
    
    
    for j in col_Names:
        if j not in features:
            
            df1=df1.drop([j],axis=1)
   
    
        
    mean_list=list(df1.mean())
    max_list = list(df1.max())
    min_list=list(df1.min())
    
    
    

 
    for row in df1.itertuples():                  
        j=0       
        
        for col in df1.columns:
            df1.at[row[0],col] = (df1.at[row[0],col] - float(mean_list[j])) / (float(max_list[j]) - float(min_list[j]))
            j+=1
  
    df2 = pd.concat([df1,y], axis=1)
    df2.to_csv(output_file)




# write_csv('train_raw.csv', 'final_train_remove_features_fre2.csv')
write_csv('dev_raw.csv', "final_dev_remove_features_fre02.csv")

                fre
count  33372.000000
mean      74.719701
std      147.744011
min    -9183.788605
25%       67.823768
50%       79.284815
75%       89.016001
max      205.372128


NameError: name 'df1' is not defined

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import model_selection, neighbors
import re
from collections import Counter
from string import punctuation
from collections import defaultdict
import datetime
from sklearn.ensemble import ExtraTreesClassifier
# load data
df = pd.read_csv("final_dev_remove_features_fre02.csv")
df1 = pd.read_csv("final_dev_remove_features_fre02.csv")
#unnamed column removed from df
df=df.drop(df.columns[0], axis=1)
df1=df1.drop(df.columns[0], axis=1)
Y_train=df['class']
X_train=df.iloc[:,:-1]
Y_test=df1['class']
X_test=df1.iloc[:,:-1]
X_train=X_train.drop(X_train.columns[[6,12,17]],axis=1)
X_test=X_test.drop(X_test.columns[[6,12,17]],axis=1)
model = ExtraTreesClassifier()
model.fit(X_train, Y_train)
print("feature importance level metrics is ",model.feature_importances_)


   len_text  capitalized_letters  ratio_capital_total  ratio_spaces_total  \
0 -0.006717            -0.003510            -0.013635            0.015943   
1  0.035621             0.009841            -0.008635            0.015943   
2 -0.047011            -0.013746            -0.022635            0.004038   
3  0.000057            -0.000840            -0.011635            0.022556   
4 -0.055304            -0.015526            -0.044635           -0.009190   

   ratio_spaces_non_spaces  ratio_capital_lower_case  ratio_numeric_total  \
0                 0.005367                 -0.000233             0.000887   
1                 0.005367                 -0.000226            -0.002113   
2                 0.000860                 -0.000244            -0.005113   
3                 0.007621                 -0.000229            -0.004113   
4                -0.003970                 -0.000271            -0.005113   

   special_chars  count_once  count_twice    ...      max_len  \
0      -0

In [6]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import scipy.stats as sps
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import neighbors
from scipy.stats import *
clf = neighbors.KNeighborsClassifier(n_neighbors=13)
clf.fit(X_train,Y_train)
acc = clf.score(X_test,Y_test)
print("accuracy is ",acc)


accuracy is  0.5154021335251109


In [7]:
#decision Tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
# %matplotlib inlin
from sklearn.metrics import accuracy_score
li=[]
#best split at 6500
c = tree.DecisionTreeClassifier(min_samples_split =6500)
features=list(X_train.columns.values)
dt = c.fit(X_train, Y_train)
Y_pred = c.predict(X_test)
score = accuracy_score(Y_test, Y_pred) * 100
print("accuracy for decision tree is ", score)




accuracy for decision tree is  58.25542370849814


In [ ]:
import graphviz
%matplotlib inline
dot_data = tree.export_graphviz(dt, out_file=None) 
graph = graphviz.Source(dot_data) 
dot_data = tree.export_graphviz(dt, out_file=None, 
                         feature_names=features,  
                         class_names= ["14-16","24-26","34-36","44-46"],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)
graph


In [ ]:
with open("fiamge.txt", "w") as f:
    f = tree.export_graphviz(c, out_file=f)


In [8]:

from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
clf = LinearSVC(random_state=0)
clf.fit(X_train, Y_train)
pred=0
count=0
for i in range(0,len(Y_test)):
    if clf.predict([X_test.iloc[1]]) == Y_test[i] :
        pred+=1
print("accuracy is ", pred/len(X_test)*100)



accuracy is  51.54320987654321
51.54320987654321
